In [69]:
import pandas as pd
from collections import defaultdict

In [84]:
data = pd.read_csv("/cellar/users/snwright/Data/Network_Analysis/Reference_Data/gtex_median_processed_1.tsv", sep="\t", index_col=0)

In [85]:
data = data[~data.Entrez.isna()]

In [86]:
dups = data[data.Entrez.duplicated()].Entrez.to_list()

In [87]:
def consolidated_genes(data):
    dups = data[data.Entrez.duplicated()].Entrez.to_list()
    dup_results = []
    for gene in dups:
        gene_data = data[data.Entrez == gene]
        consolidated_data = {}
        for tissue in gene_data.columns[2:-1]:
            if min (gene_data[tissue]) > 0.0001:
                consolidated_data[tissue] = gene_data[tissue].mean()
            elif max(gene_data[tissue]) < 0.0001:
                consolidated_data[tissue] = gene_data[tissue].mean()
            else:
                consolidated_data[tissue] = max(gene_data[tissue])
        summary = {}
        if len(gene_data.Ensembl_ID.unique()) == 1:
            summary['Ensembl_ID'] = gene_data.Ensembl_ID.unique()
        else:
            ids = [x.split(".")[0] for x in gene_data.Ensembl_ID.unique()]
            if ids[0] == ids[1]:
                summary['Ensembl_ID'] = ids[0]
            else:
                summary['Ensembl_ID'] = "_".join(gene_data.Ensembl_ID.unique())
        if len(gene_data.Symbol.unique()) == 1:
            summary['Symbol'] = gene_data.Symbol.unique()
        else:
            summary['Symbol'] = "_".join(gene_data.Symbol.unique())
        dup_results.append(pd.DataFrame({**summary, **consolidated_data, "Entrez":gene}, index=[gene]))
    all_results = pd.concat(dup_results)
    out_data = data[~data.Entrez.isin(dups)]
    return pd.concat([out_data, all_results]).reset_index(drop=True)

In [88]:
data = consolidated_genes(data)

In [72]:
tissue_categories = defaultdict(list)
for tissue in data.columns[2:-1]:
    tissue_categories[tissue.split(" - ")[0]].append(tissue)
tissue_categories['Cardiovascular'] = tissue_categories['Heart'] + tissue_categories['Artery']
tissue_categories['Digestive'] = tissue_categories['Esophagus'] + tissue_categories['Stomach'] + tissue_categories['Small Intestine'] + tissue_categories['Colon']

In [107]:
def get_gene_expression(data, gene, gene_idx):
    return data.loc[gene_idx[gene], data.columns[2:-1]]

def get_tissue_expression(data, gene, gene_idx, tissue, tissue_dict):
    gene_data = get_gene_expression(data, gene, gene_idx)
    if type(tissue) == str:
        return gene_data[tissue_dict[tissue]]
    else:
        tissue_types = []
        for x in tissue:
            tissue_types += tissue_dict[x]
        return gene_data.loc[tissue_types]

def global_expression_median(data, gene, gene_idx):
    gene_data = get_gene_expression(data, gene, gene_idx)
    return gene_data.median()

def tissue_expression_median(data, gene ,gene_idx, tissue, tissue_dict):
    gene_data = get_tissue_expression(data, gene, gene_idx, tissue, tissue_dict)
    return gene_data.median()
    

In [90]:
gene_idx = data.loc[:, 'Entrez'].to_dict()
# reverse this dictionary
gene_idx = {v: k for k, v in gene_idx.items()}

In [108]:
tissue_expression_median(data, 8490, gene_idx, ['Brain', 'Heart'], tissue_categories)

32.0361405

In [35]:
gene = 8490.0

In [21]:
dup_data.head()

,Ensembl_ID,Symbol,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Bladder,Brain - Amygdala,...,Skin - Sun Exposed (Lower leg),Small Intestine - Terminal Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole Blood,Entrez
1945,ENSG00000231364.2,LINC01712,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,7.33016,0.000000,0.000000,0.000000,0.000000,105375358.0
3458,ENSG00000143248.12,RGS5,182.064000,141.974000,48.910100,1458.190000,424.493000,2303.670000,149.327000,56.553800,...,30.511500,63.202300,13.745200,75.600300,31.70810,290.534000,111.506000,63.293100,0.368054,8490.0
3461,ENSG00000232995.7,RGS5,0.423776,0.364069,0.622097,0.494969,0.555716,0.452362,0.752952,0.319591,...,0.458926,0.736113,0.962067,0.558313,3.39578,1.321580,0.767546,0.623761,0.065136,8490.0
3761,ENSG00000243062.5,RP11-12M5.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,43.19510,0.000000,0.000000,0.000000,0.000000,148753.0
3762,ENSG00000143340.6,FAM163A,0.000000,0.280420,0.124643,0.032412,0.030901,0.000000,0.055593,0.467590,...,0.000000,1.589930,0.101426,0.539381,33.79520,0.896442,0.041007,0.030848,0.000000,148753.0
